In [1]:
import pandas as pd
import csv
import re
from stop_words import get_stop_words

In [2]:
train_data = pd.read_csv('train.csv')
train_title = train_data['title'].tolist()
test_data = pd.read_csv('test.csv')
test_title = test_data['title'].tolist()

In [3]:
# incomplete
irrelevant_words = ["wa", "murah", "termurah", "original", "satuan", "ready" ,"stok", "stock", "limited", "edition", "promo", "bulan", "ini", "hari", "minggu", "gratis", "ongkir", "garansi", "pcs", "pc", "ori", "berkualitas", "new", "tanpa", "beli", "terlaris", "terbaru", "baru", "gudang", "bayar", "free", "harga", "chat", "seller", "terbatas", "premium", "jual", "paket", "minat", "khusus", "exclusive", "eksklusif", "asli", "barang", "terbaik", "laris", "distributor", "product", "bonus", "bagus", "dijual", "terjamin", "whatsapp", "serius", "real", "like", "price", "terkini", "order", "pengiriman", "info", "silahkan", "silakan", "setengah", "via", "impor", "import", "diskon", "discount", "lengkap", "terupdate", "bigsale", "obral", "dijamin", "terbukti", "discon", "dapat", "tidak", "terakhir", "buruan", "pemesanan", "pesanan", "pesan", "buy", "banget", "pilihan", "serba", "non", "ecer", "oke", "ok", "amazing", "banyak", "dus", "wholesale", "update", "updet", "bukan", "from", "tutup", "orginal", "siap", "arrival", "pre", "kirim", "habisin", "extreme", "hadiah", "selling", "kualitasnya", "boskuu", "dicari", "better", "exlusive", "palsu", "kak", "deal", "buat", "dapatkan", "tersedia", "aja", "sebelum", "terpercaya", "pasti", "langsung", "12.12", "edisi", "no.1", "seluruh", "hello", "gila", "gilaa", "variasi", "jaminan", "dapetin", "wow", "cocok", "istimewa", "recommended", "recomended", "menyambut", "masyarakat", "restock", "tren", "datang", "now", "terlengkap", "produck", "mantap", "kembali", "terjangkau", "nego", "hubungi", "berubah", "terkeren", "seru", "terbuka", "terpopuler", "sold", "cuma", "unggulan", "satuan", "very", "fake", "mesin", "geratis", "besar2an", "ekslusif", "mudah", "koleksi", "gojek", "salee", "pasang", "gransi", "packing", "pack", "label", "miliki", "imitasi", "pilih", "chating", "grtis", "suplier", "supplier", "semua", "indah", "ended", "bagian", "komplit", "klasik", "happy", "ajaib", "jamin", "nasional", "cheap", "hub", "cheapest", "eceran"]
irrelevant_words.extend(get_stop_words('id'))
irrelevant_words

['wa',
 'murah',
 'termurah',
 'original',
 'satuan',
 'ready',
 'stok',
 'stock',
 'limited',
 'edition',
 'promo',
 'bulan',
 'ini',
 'hari',
 'minggu',
 'gratis',
 'ongkir',
 'garansi',
 'pcs',
 'pc',
 'ori',
 'berkualitas',
 'new',
 'tanpa',
 'beli',
 'terlaris',
 'terbaru',
 'baru',
 'gudang',
 'bayar',
 'free',
 'harga',
 'chat',
 'seller',
 'terbatas',
 'premium',
 'jual',
 'paket',
 'minat',
 'khusus',
 'exclusive',
 'eksklusif',
 'asli',
 'barang',
 'terbaik',
 'laris',
 'distributor',
 'product',
 'bonus',
 'bagus',
 'dijual',
 'terjamin',
 'whatsapp',
 'serius',
 'real',
 'like',
 'price',
 'terkini',
 'order',
 'pengiriman',
 'info',
 'silahkan',
 'silakan',
 'setengah',
 'via',
 'impor',
 'import',
 'diskon',
 'discount',
 'lengkap',
 'terupdate',
 'bigsale',
 'obral',
 'dijamin',
 'terbukti',
 'discon',
 'dapat',
 'tidak',
 'terakhir',
 'buruan',
 'pemesanan',
 'pesanan',
 'pesan',
 'buy',
 'banget',
 'pilihan',
 'serba',
 'non',
 'ecer',
 'oke',
 'ok',
 'amazing',
 'bany

In [4]:
def delete_words(matchobj):
    word = matchobj.group(0)
    if word in irrelevant_words:
        return ""
    else:
        return word

In [18]:
def preprocessing(title_array):
    new_title_array = []
    units = ['ml', 'gr', 'gram', 'gb', 'mg', 'g', 'oz', 'cc', 'inch', 'mp', 'm', 'cm', 'kg']
    # temporary pattern
    units_pattern = re.compile('((\A| )\d+) +((ml)|(gr)|(gram)|(gb)|(mg)|(g)|(oz)|(cc)|(inch)|(mp)|(m)|(cm)|(kg)( |\Z))')
    for title in title_array:
        # lowercase and remove special characters
        new_title = re.sub('[^a-z0-9 ]', '', title.lower()) 
        # remove numbers with 5 or more digits
        new_title = re.sub('\d{5,}', '', new_title)
        # remove irrelevant words (incomplete list)
        new_title = re.sub('[a-z]+', delete_words, new_title)
        # combine numbers and units (e.g. 3 ml -> 3ml)
        new_title = units_pattern.sub(r'\1\3', new_title)
        # remove 1-letter words
        new_title = re.sub('(\A| )[a-z]{1}(?= )', '', new_title)
        # remove extra space character
        new_title = re.sub(' +', ' ', new_title)
        # strip trailing space characters
        new_title_array.append(new_title.strip())
    return new_title_array

In [19]:
new_train_title = preprocessing(train_title)
new_test_title = preprocessing(test_title)

In [28]:
# check for combined words and numbers
keywords = []
pattern = re.compile("\d+[a-z]+")
for title in new_train_title:
    occurence = pattern.findall(title)
    if occurence:
        keywords.extend(occurence)

In [27]:
sorted_keywords = sorted(keywords, key=lambda x: sum(c.isalpha() for c in x), reverse=True)

In [29]:
[[keyword] for keyword in sorted_keywords if sum(c.isalpha() for c in keyword) == 2]

[['20ml'],
 ['100ml'],
 ['2in'],
 ['15gx'],
 ['2in'],
 ['35ml'],
 ['50ml'],
 ['20ml'],
 ['2in'],
 ['80ml'],
 ['50gr'],
 ['25gr'],
 ['5in'],
 ['15gr'],
 ['2in'],
 ['017oz'],
 ['30gr'],
 ['50ml'],
 ['9gr'],
 ['40ml'],
 ['40gr'],
 ['1ml'],
 ['35gr'],
 ['3ce'],
 ['200ml'],
 ['30gr'],
 ['14gr'],
 ['2in'],
 ['30gr'],
 ['028oz'],
 ['30ml'],
 ['30ml'],
 ['50gr'],
 ['150ml'],
 ['30ml'],
 ['2in'],
 ['50pa'],
 ['7ml'],
 ['160ml'],
 ['30gr'],
 ['40ml'],
 ['5gr'],
 ['15gr'],
 ['60ml'],
 ['14gr'],
 ['5gr'],
 ['30ml'],
 ['20ml'],
 ['30ml'],
 ['15ml'],
 ['20ml'],
 ['30pa'],
 ['10gr'],
 ['25gr'],
 ['5ml'],
 ['50ml'],
 ['40ml'],
 ['10ml'],
 ['30gr'],
 ['20ml'],
 ['30ml'],
 ['12ml'],
 ['50ml'],
 ['20gr'],
 ['500ml'],
 ['5ml'],
 ['27pa'],
 ['15gr'],
 ['50ml'],
 ['20ml'],
 ['300ml'],
 ['10ml'],
 ['20ml'],
 ['30gr'],
 ['500gr'],
 ['8gr'],
 ['5ml'],
 ['5gr'],
 ['24gr'],
 ['30ea'],
 ['50gr'],
 ['430rb'],
 ['50ml'],
 ['3ce'],
 ['75ml'],
 ['300ml'],
 ['300ml'],
 ['15ml'],
 ['2in'],
 ['10ml'],
 ['149rb'],
 ['3ml

In [30]:
train_data['title'] = new_train_title
test_data['title'] = new_test_title
train_data.to_csv(index=False, path_or_buf='new_train.csv')
test_data.to_csv(index=False, path_or_buf='new_test.csv')